In [2]:
import requests
import pandas as pd
import os 
import json
import http.client
import time
import numpy as np

In [5]:
season = 2021

players = pd.read_csv('C:/coding/Python/practice/football_analysis/2nd analysis - underestimated players/best_underdog_players_season_{}.csv'.format(season))
players = players.dropna()
players['player_id'] = players['player_id'].astype(int)
players

,team_id,team_name,league_id,league_name,num_underdog,avg_rating,player_id,player_name,position
0,542,Alaves,140,La Liga,18,7.505556,18907,Joselu,F
1,511,Empoli,135,Serie A,12,8.041667,31354,Guglielmo Vicario,G
2,41,Southampton,39,Premier League,12,7.666667,2938,James Ward-Prowse,M
3,188,Arminia Bielefeld,78,Bundesliga 1,10,7.860000,25004,Stefan Ortega,G
4,53,Reading,40,Championship,10,7.820000,19619,John Swift,M
...,...,...,...,...,...,...,...,...,...
3582,221,Belenenses,94,Primeira Liga,1,0.000000,281149,Henrique Pires,D
3583,221,Belenenses,94,Primeira Liga,1,0.000000,163090,João Monteiro,D
3584,221,Belenenses,94,Primeira Liga,1,0.000000,282721,Andre Lopes,D
3585,221,Belenenses,94,Primeira Liga,1,0.000000,279445,Álvaro Ramalho,G


In [172]:
# transfers in the year after the beginning of the season - most of them should be after end of the season

player_list = players['player_id'].values.tolist()
transfer = pd.DataFrame()

for player in player_list:
    
    conn = http.client.HTTPSConnection("v3.football.api-sports.io")

    headers = {
        'x-rapidapi-host': "v3.football.api-sports.io",
        'x-rapidapi-key': "(my account)"
        }

    conn.request("GET", "/transfers?player={}".format(player), headers=headers)

    res = conn.getresponse()
    data = res.read()

    json_transfer = json.loads(data.decode("utf-8"))
    df_transfer = pd.DataFrame(json_transfer['response'])
    
    try:
        
        transfer_year = df_transfer['transfers'][0][0]['date'][0:4]

        if transfer_year == str(season + 1):

            transfer = transfer.append(pd.DataFrame([
                [df_transfer['player'][0]['id'],
                 df_transfer['player'][0]['name'],
                 df_transfer['transfers'][0][0]['date'],
                 df_transfer['transfers'][0][0]['type'],
                 df_transfer['transfers'][0][0]['teams']['in']['id'],
                 df_transfer['transfers'][0][0]['teams']['in']['name']]],
                 columns=['player_id', 'player_name', 'date', 'type', 'new_team_id', 'new_team_name']))
        else:
            continue
    
    except KeyError:
        continue
            


In [173]:
transfer

,player_id,player_name,date,type,new_team_id,new_team_name
0,31354,G. Vicario,2022-07-01,€ 8.5M,511,Empoli
0,25004,S. Ortega Moreno,2022-07-01,Free,50,Manchester City
0,19619,J. Swift,2022-07-01,Free,60,West Brom
0,10497,Lincoln Henrique Oliveira dos Santos,2022-06-17,€ 3.5M,611,Fenerbahce
0,47291,Gorka Guruzeta,2022-07-03,Free,531,Athletic Club
...,...,...,...,...,...,...
0,7649,D. Nemeth,2022-07-01,€ 1.3M,186,FC St. Pauli
0,182167,Adrián Butzke,2022-07-01,N/A,715,Granada CF
0,1760,Glenn Bell Dollar Middleton,2022-06-01,N/A,257,Rangers
0,19002,Josh Murphy,2022-07-01,N/A,43,Cardiff


In [184]:
transfer = transfer.drop_duplicates()
transfer

,player_id,player_name,date,type,new_team_id,new_team_name
0,31354,G. Vicario,2022-07-01,€ 8.5M,511,Empoli
0,25004,S. Ortega Moreno,2022-07-01,Free,50,Manchester City
0,19619,J. Swift,2022-07-01,Free,60,West Brom
0,10497,Lincoln Henrique Oliveira dos Santos,2022-06-17,€ 3.5M,611,Fenerbahce
0,47291,Gorka Guruzeta,2022-07-03,Free,531,Athletic Club
...,...,...,...,...,...,...
0,30812,Ignacio Pussetto,2022-07-01,N/A,38,Watford
0,19072,Todd Cantwell,2022-07-01,N/A,71,Norwich
0,7649,D. Nemeth,2022-07-01,€ 1.3M,186,FC St. Pauli
0,182167,Adrián Butzke,2022-07-01,N/A,715,Granada CF


In [185]:
# use parameter encoding='utf-8-sig' to be able to display letters with accents etc with no error
transfer.to_csv('C:/coding/Python/practice/football_analysis/2nd analysis - underestimated players/transfer_season_{}.csv'.format(season), index=False, encoding='utf-8-sig')

In [171]:
# player age data

player_list = players['player_id'].values.tolist()

age = pd.DataFrame()

no_age_data = []

for player in player_list:

    conn = http.client.HTTPSConnection("v3.football.api-sports.io")

    headers = {
        'x-rapidapi-host': "v3.football.api-sports.io",
        'x-rapidapi-key': "(my account)"
        }

    conn.request("GET", "/players?id={}&season={}".format(player, season), headers=headers)

    res = conn.getresponse()
    data = res.read()

    json_data = json.loads(data.decode("utf-8"))
    df_age = pd.DataFrame(json_data['response'])

    try:
    
        age = age.append(pd.DataFrame([
            [df_age['player'][0]['id'],
             df_age['player'][0]['name'],
             df_age['player'][0]['age']]],
             columns=['player_id', 'player_name', 'age']))
    
    except KeyError:
        no_age_data.append(player)
        continue
    
age['age'] = age['age'].astype(int)
no_age_data

[287792, 322979, 281489, 7217, 322984, 178913]

In [178]:
age

,player_id,player_name,age
0,18907,Joselu,32
0,31354,G. Vicario,26
0,2938,J. Ward-Prowse,28
0,25004,S. Ortega Moreno,30
0,19619,J. Swift,27
...,...,...,...
0,281149,Henrique Ribeiro Pires,20
0,127554,Eduardo de Sousa Santos,23
0,279445,Álvaro Raul Sofrimento Ramalho,23
0,294770,António Luís Cavaco Silva Sá Montez,21


In [182]:
age = age.drop_duplicates()
age

,player_id,player_name,age
0,18907,Joselu,32
0,31354,G. Vicario,26
0,2938,J. Ward-Prowse,28
0,25004,S. Ortega Moreno,30
0,19619,J. Swift,27
...,...,...,...
0,281149,Henrique Ribeiro Pires,20
0,127554,Eduardo de Sousa Santos,23
0,279445,Álvaro Raul Sofrimento Ramalho,23
0,294770,António Luís Cavaco Silva Sá Montez,21


In [183]:
age.to_csv('C:/coding/Python/practice/football_analysis/2nd analysis - underestimated players/age_season_{}.csv'.format(season), index=False, encoding='utf-8-sig')

In [138]:
# standings at the end of the season

leagues = pd.read_csv('C:/coding/Python/practice/football_analysis/2nd analysis - underestimated players/df_leagues.csv')
league_list = leagues['id'].values.tolist()

standings = pd.DataFrame()

for league in league_list:

    conn = http.client.HTTPSConnection("v3.football.api-sports.io")

    headers = {
        'x-rapidapi-host': "v3.football.api-sports.io",
        'x-rapidapi-key': "(my account)"
        }

    conn.request("GET", "/standings?league={}&season={}".format(league, season), headers=headers)

    res = conn.getresponse()
    data = res.read()

    json_data = json.loads(data.decode("utf-8"))
    df_standings = pd.DataFrame(json_data['response'])

    # get the standings only for regular rounds, not play-offs
        
    dicts = {}
    for i in range(len(df_standings['league'][0]['standings'])):
        dicts[i] = len(df_standings['league'][0]['standings'][i])

    section = list(dicts.values()).index(max(list(dicts.values())))
    num_teams = range(len(df_standings['league'][0]['standings'][section]))
    
    for team in num_teams:

        standings = standings.append(pd.DataFrame([
            [df_standings['league'][0]['standings'][section][team]['team']['id'],
             df_standings['league'][0]['standings'][section][team]['team']['name'],
             df_standings['league'][0]['id'],
             season,
             df_standings['league'][0]['standings'][section][team]['rank']]],
             columns=['team_id', 'team_name', 'league_id', 'season', 'rank']))
    
standings['rank'] = standings['rank'].astype(int)
standings.to_csv('C:/coding/Python/practice/football_analysis/2nd analysis - underestimated players/standings_season_{}.csv'.format(season), index=False, encoding='utf-8-sig')

In [141]:
standings

,team_id,team_name,league_id,season,rank
0,50,Manchester City,39,2021,1
0,40,Liverpool,39,2021,2
0,49,Chelsea,39,2021,3
0,47,Tottenham,39,2021,4
0,42,Arsenal,39,2021,5
...,...,...,...,...,...
0,1084,Ural,235,2021,12
0,1994,Khimki,235,2021,13
0,1078,FC UFA,235,2021,14
0,1083,Rubin,235,2021,15
